# Neural Machine Translation

In [1]:
### Import modules
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


## 1 - Translating human readable dates into machine readable dates

In [2]:
### Set up fake module instance
fake = Faker()
fake.seed(12345)
random.seed(12345)

# Define format of the data we would like to generate
FORMATS = ['short','medium','long','full','d MMM YYY', 'd MMMM YYY','dd MMM YYY','d MMM, YYY',
           'd MMMM, YYY','dd, MMM YYY','d MM YY','d MMMM YYY','MMMM d YYY','MMMM d, YYY','dd.MM.YY']

In [3]:
### Select 10,000 samples
m = 10000

### Set up outputs   
human_vocab2 = set()
machine_vocab2 = set()
dataset = []
Tx = 30

for i in tqdm(range(m)):

    dt = fake.date_object()

    human_readable1 = format_date(dt, format=random.choice(FORMATS),  locale='en_US') 
    human_readable2 = human_readable1.lower()
    human_readable = human_readable2.replace(',','')
    machine_readable = dt.isoformat()
    
    if human_readable is not None:
        dataset.append((human_readable, machine_readable))
        human_vocab2.update(tuple(human_readable))
        machine_vocab2.update(tuple(machine_readable))
    
human_vocab = dict(zip(sorted(human_vocab2) + ['<unk>', '<pad>'], 
                     list(range(len(human_vocab2) + 2))))
inv_machine_vocab = dict(enumerate(sorted(machine_vocab2)))
machine_vocab = {v:k for k,v in inv_machine_vocab.items()}


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 13613.02it/s]


In [4]:
dataset[:10]

[('9 may 1998', '1998-05-09'),
 ('10.09.70', '1970-09-10'),
 ('4/28/90', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26'),
 ('monday march 7 1983', '1983-03-07'),
 ('sunday may 22 1988', '1988-05-22'),
 ('tuesday july 8 2008', '2008-07-08'),
 ('08 sep 1999', '1999-09-08'),
 ('1 jan 1981', '1981-01-01'),
 ('monday may 22 1995', '1995-05-22')]

In [5]:
def string_to_int(string, length, vocab):

    #make lower to standardize
    string = string.lower()
    string = string.replace(',','')
    
    if len(string) > length:
        string = string[:length]
        
    rep = list(map(lambda x: vocab.get(x, '<unk>'), string))
    
    if len(string) < length:
        rep += [vocab['<pad>']] * (length - len(string))
    
    return rep


In [6]:
### Set up X and Y input length
Tx = 30
Ty = 10
    
X1, Y1 = zip(*dataset)
    
X = np.array([string_to_int(i, Tx, human_vocab) for i in X1])
Y2 = [string_to_int(t, Ty, machine_vocab) for t in Y1]
    
Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y2)))

Y = np.array(Y2)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


## 2 - Neural machine translation with attention


In [7]:
def softmax(x, axis=1):
 
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s

In [8]:
### Define size of a and s states
n_a = 32
n_s = 64

### Build Keras Recurrent Neural Network Model

human_vocab_size = len(human_vocab)
machine_vocab_size = len(machine_vocab)

# Define the inputs of your model with a shape (Tx,)
# Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
X = Input(shape=(Tx, human_vocab_size), name= 'X')
s0 = Input(shape=(n_s,), name='s0')
c0 = Input(shape=(n_s,), name='c0')
s = s0
c = c0
    
# Initialize empty list of outputs
outputs = []

# Define your pre-attention Bi-LSTM. Remember to use return_sequences=True.
a = Bidirectional(LSTM(n_a, return_sequences=True, name='bidirectional_1'), merge_mode='concat')(X)
    
# Iterate for Ty steps
for t in range(Ty):
    
    # Perform one step of the attention mechanism to get back the context vector at step t
    
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" 
    s_prev1 = RepeatVector(Tx)(s)
   
    # Use concatenator to concatenate a and s_prev on the last axis 
    concat = Concatenate(axis=-1)([a, s_prev1])
    
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e   
    e = Dense(10, activation = "tanh")(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. 
    energies = Dense(1, activation = "relu")(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" 
    alphas = Activation(softmax)(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell 
    context = Dot(axes = 1)([alphas, a])
        
    # Apply the post-attention LSTM cell to the "context" vector.
    s, _, c = LSTM(n_s, return_state = True)(context, initial_state = [s, c])
        
    # Apply Dense layer to the hidden state output of the post-attention LSTM
    out = Dense(len(machine_vocab), activation=softmax)(s)
    
    # Append "out" to the "outputs" list
    outputs.append(out)
    
# Create model instance taking three inputs and returning the list of outputs.
model = Model(inputs=(X, s0, c0), outputs=outputs)
    

In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X (InputLayer)                  (None, 30, 37)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 64)       17920       X[0][0]                          
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 30, 64)       0           s0[0][0]                         
__________________________________________________________________________________________________
concatenat

__________________________________________________________________________________________________
repeat_vector_5 (RepeatVector)  (None, 30, 64)       0           lstm_4[0][0]                     
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 30, 128)      0           bidirectional_1[0][0]            
                                                                 repeat_vector_5[0][0]            
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 30, 10)       1290        concatenate_5[0][0]              
__________________________________________________________________________________________________
dense_14 (Dense)                (None, 30, 1)        11          dense_13[0][0]                   
__________________________________________________________________________________________________
activation

activation_9 (Activation)       (None, 30, 1)        0           dense_26[0][0]                   
__________________________________________________________________________________________________
dot_9 (Dot)                     (None, 1, 64)        0           activation_9[0][0]               
                                                                 bidirectional_1[0][0]            
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, 64), (None,  33024       dot_9[0][0]                      
                                                                 lstm_8[0][0]                     
                                                                 lstm_8[0][2]                     
__________________________________________________________________________________________________
repeat_vector_10 (RepeatVector) (None, 30, 64)       0           lstm_9[0][0]                     
__________

In [10]:
### Set up Keras Optimizers
from keras import optimizers

opt = optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [11]:
### Set up input parameter as 0
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))

### Prepare train Y data
outputs = list(Yoh.swapaxes(0,1))

In [12]:
model.fit([Xoh, s0, c0], outputs, epochs=10, batch_size=100)

Epoch 1/10


 1700/10000 [====>.........................] - ETA: 24:37 - loss: 24.0003 - dense_3_loss: 2.3931 - dense_6_loss: 2.4162 - dense_9_loss: 2.3896 - dense_12_loss: 2.3980 - dense_15_loss: 2.4144 - dense_18_loss: 2.4091 - dense_21_loss: 2.3989 - dense_24_loss: 2.3960 - dense_27_loss: 2.3876 - dense_30_loss: 2.3974 - dense_3_acc: 0.3800 - dense_6_acc: 0.0000e+00 - dense_9_acc: 0.2900 - dense_12_acc: 0.1100 - dense_15_acc: 0.0000e+00 - dense_18_acc: 0.0000e+00 - dense_21_acc: 0.0100 - dense_24_acc: 0.0000e+00 - dense_27_acc: 0.3100 - dense_30_acc: 0.090 - ETA: 12:20 - loss: 23.9571 - dense_3_loss: 2.3906 - dense_6_loss: 2.4108 - dense_9_loss: 2.3894 - dense_12_loss: 2.3981 - dense_15_loss: 2.4043 - dense_18_loss: 2.4006 - dense_21_loss: 2.3973 - dense_24_loss: 2.3833 - dense_27_loss: 2.3854 - dense_30_loss: 2.3974 - dense_3_acc: 0.3800 - dense_6_acc: 0.0050 - dense_9_acc: 0.2350 - dense_12_acc: 0.1050 - dense_15_acc: 0.0500 - dense_18_acc: 0.0000e+00 - dense_21_acc: 0.0600 - dense_24_acc: 0.4

 3400/10000 [=========>....................] - ETA: 1:23 - loss: 22.6635 - dense_3_loss: 2.3173 - dense_6_loss: 2.2929 - dense_9_loss: 2.3399 - dense_12_loss: 2.3908 - dense_15_loss: 2.1433 - dense_18_loss: 2.1733 - dense_21_loss: 2.3818 - dense_24_loss: 1.9902 - dense_27_loss: 2.2486 - dense_30_loss: 2.3855 - dense_3_acc: 0.3783 - dense_6_acc: 0.5039 - dense_9_acc: 0.2211 - dense_12_acc: 0.1022 - dense_15_acc: 0.8589 - dense_18_acc: 0.6333 - dense_21_acc: 0.1378 - dense_24_acc: 0.9433 - dense_27_acc: 0.3150 - dense_30_acc: 0.08 - ETA: 1:18 - loss: 22.4928 - dense_3_loss: 2.3093 - dense_6_loss: 2.2789 - dense_9_loss: 2.3339 - dense_12_loss: 2.3904 - dense_15_loss: 2.1089 - dense_18_loss: 2.1460 - dense_21_loss: 2.3793 - dense_24_loss: 1.9363 - dense_27_loss: 2.2267 - dense_30_loss: 2.3830 - dense_3_acc: 0.3763 - dense_6_acc: 0.5121 - dense_9_acc: 0.2189 - dense_12_acc: 0.1011 - dense_15_acc: 0.8663 - dense_18_acc: 0.6337 - dense_21_acc: 0.1405 - dense_24_acc: 0.9463 - dense_27_acc: 0.3

 5100/10000 [==============>...............] - ETA: 40s - loss: 18.9295 - dense_3_loss: 2.0144 - dense_6_loss: 1.8325 - dense_9_loss: 2.1450 - dense_12_loss: 2.3710 - dense_15_loss: 1.3156 - dense_18_loss: 1.5629 - dense_21_loss: 2.3500 - dense_24_loss: 1.1182 - dense_27_loss: 1.8557 - dense_30_loss: 2.3640 - dense_3_acc: 0.4200 - dense_6_acc: 0.5791 - dense_9_acc: 0.2046 - dense_12_acc: 0.0994 - dense_15_acc: 0.9274 - dense_18_acc: 0.6866 - dense_21_acc: 0.1514 - dense_24_acc: 0.9709 - dense_27_acc: 0.3254 - dense_30_acc: 0.094 - ETA: 38s - loss: 18.7432 - dense_3_loss: 1.9860 - dense_6_loss: 1.8023 - dense_9_loss: 2.1316 - dense_12_loss: 2.3696 - dense_15_loss: 1.2793 - dense_18_loss: 1.5356 - dense_21_loss: 2.3482 - dense_24_loss: 1.0872 - dense_27_loss: 1.8408 - dense_30_loss: 2.3625 - dense_3_acc: 0.4272 - dense_6_acc: 0.5819 - dense_9_acc: 0.2056 - dense_12_acc: 0.0986 - dense_15_acc: 0.9294 - dense_18_acc: 0.6883 - dense_21_acc: 0.1525 - dense_24_acc: 0.9717 - dense_27_acc: 0.32

 6800/10000 [===================>..........] - ETA: 23s - loss: 16.6282 - dense_3_loss: 1.6131 - dense_6_loss: 1.4624 - dense_9_loss: 1.9814 - dense_12_loss: 2.3543 - dense_15_loss: 0.8862 - dense_18_loss: 1.2424 - dense_21_loss: 2.3257 - dense_24_loss: 0.7530 - dense_27_loss: 1.6607 - dense_30_loss: 2.3490 - dense_3_acc: 0.4856 - dense_6_acc: 0.5812 - dense_9_acc: 0.2048 - dense_12_acc: 0.0973 - dense_15_acc: 0.9512 - dense_18_acc: 0.7085 - dense_21_acc: 0.1527 - dense_24_acc: 0.9804 - dense_27_acc: 0.3294 - dense_30_acc: 0.105 - ETA: 22s - loss: 16.5361 - dense_3_loss: 1.5957 - dense_6_loss: 1.4475 - dense_9_loss: 1.9747 - dense_12_loss: 2.3533 - dense_15_loss: 0.8695 - dense_18_loss: 1.2305 - dense_21_loss: 2.3237 - dense_24_loss: 0.7388 - dense_27_loss: 1.6540 - dense_30_loss: 2.3484 - dense_3_acc: 0.4879 - dense_6_acc: 0.5817 - dense_9_acc: 0.2047 - dense_12_acc: 0.0981 - dense_15_acc: 0.9521 - dense_18_acc: 0.7089 - dense_21_acc: 0.1538 - dense_24_acc: 0.9808 - dense_27_acc: 0.32

...

 3400/10000 [=========>....................] - ETA: 15s - loss: 6.0392 - dense_3_loss: 0.0532 - dense_6_loss: 0.0501 - dense_9_loss: 0.8845 - dense_12_loss: 1.8179 - dense_15_loss: 3.4167e-04 - dense_18_loss: 0.0599 - dense_21_loss: 0.5778 - dense_24_loss: 3.3024e-04 - dense_27_loss: 0.7115 - dense_30_loss: 1.8837 - dense_3_acc: 0.9822 - dense_6_acc: 0.9806 - dense_9_acc: 0.5533 - dense_12_acc: 0.2956 - dense_15_acc: 1.0000 - dense_18_acc: 0.9756 - dense_21_acc: 0.8322 - dense_24_acc: 1.0000 - dense_27_acc: 0.7550 - dense_30_acc: 0.31 - ETA: 15s - loss: 6.0472 - dense_3_loss: 0.0533 - dense_6_loss: 0.0504 - dense_9_loss: 0.8869 - dense_12_loss: 1.8204 - dense_15_loss: 3.4335e-04 - dense_18_loss: 0.0609 - dense_21_loss: 0.5801 - dense_24_loss: 3.3085e-04 - dense_27_loss: 0.7121 - dense_30_loss: 1.8824 - dense_3_acc: 0.9821 - dense_6_acc: 0.9800 - dense_9_acc: 0.5500 - dense_12_acc: 0.2937 - dense_15_acc: 1.0000 - dense_18_acc: 0.9758 - dense_21_acc: 0.8321 - dense_24_acc: 1.0000 - dense

 5100/10000 [==============>...............] - ETA: 12s - loss: 6.0070 - dense_3_loss: 0.0516 - dense_6_loss: 0.0490 - dense_9_loss: 0.8849 - dense_12_loss: 1.8483 - dense_15_loss: 3.4700e-04 - dense_18_loss: 0.0586 - dense_21_loss: 0.5829 - dense_24_loss: 3.3467e-04 - dense_27_loss: 0.6902 - dense_30_loss: 1.8408 - dense_3_acc: 0.9811 - dense_6_acc: 0.9789 - dense_9_acc: 0.5463 - dense_12_acc: 0.2900 - dense_15_acc: 1.0000 - dense_18_acc: 0.9766 - dense_21_acc: 0.8320 - dense_24_acc: 1.0000 - dense_27_acc: 0.7634 - dense_30_acc: 0.32 - ETA: 12s - loss: 6.0073 - dense_3_loss: 0.0524 - dense_6_loss: 0.0497 - dense_9_loss: 0.8842 - dense_12_loss: 1.8485 - dense_15_loss: 3.4502e-04 - dense_18_loss: 0.0589 - dense_21_loss: 0.5832 - dense_24_loss: 3.3458e-04 - dense_27_loss: 0.6891 - dense_30_loss: 1.8405 - dense_3_acc: 0.9811 - dense_6_acc: 0.9789 - dense_9_acc: 0.5464 - dense_12_acc: 0.2900 - dense_15_acc: 1.0000 - dense_18_acc: 0.9764 - dense_21_acc: 0.8317 - dense_24_acc: 1.0000 - dense

 6800/10000 [===================>..........] - ETA: 9s - loss: 5.9603 - dense_3_loss: 0.0536 - dense_6_loss: 0.0509 - dense_9_loss: 0.8830 - dense_12_loss: 1.8340 - dense_15_loss: 3.3032e-04 - dense_18_loss: 0.0604 - dense_21_loss: 0.5764 - dense_24_loss: 3.3198e-04 - dense_27_loss: 0.6801 - dense_30_loss: 1.8212 - dense_3_acc: 0.9806 - dense_6_acc: 0.9787 - dense_9_acc: 0.5483 - dense_12_acc: 0.2965 - dense_15_acc: 1.0000 - dense_18_acc: 0.9738 - dense_21_acc: 0.8313 - dense_24_acc: 1.0000 - dense_27_acc: 0.7692 - dense_30_acc: 0.326 - ETA: 9s - loss: 5.9520 - dense_3_loss: 0.0532 - dense_6_loss: 0.0504 - dense_9_loss: 0.8823 - dense_12_loss: 1.8318 - dense_15_loss: 3.2949e-04 - dense_18_loss: 0.0604 - dense_21_loss: 0.5747 - dense_24_loss: 3.3247e-04 - dense_27_loss: 0.6797 - dense_30_loss: 1.8189 - dense_3_acc: 0.9808 - dense_6_acc: 0.9787 - dense_9_acc: 0.5494 - dense_12_acc: 0.2987 - dense_15_acc: 1.0000 - dense_18_acc: 0.9740 - dense_21_acc: 0.8323 - dense_24_acc: 1.0000 - dense_

 8500/10000 [========================>.....] - ETA: 6s - loss: 5.8747 - dense_3_loss: 0.0527 - dense_6_loss: 0.0497 - dense_9_loss: 0.8762 - dense_12_loss: 1.8251 - dense_15_loss: 3.1514e-04 - dense_18_loss: 0.0577 - dense_21_loss: 0.5627 - dense_24_loss: 3.3262e-04 - dense_27_loss: 0.6606 - dense_30_loss: 1.7895 - dense_3_acc: 0.9819 - dense_6_acc: 0.9800 - dense_9_acc: 0.5587 - dense_12_acc: 0.2996 - dense_15_acc: 1.0000 - dense_18_acc: 0.9757 - dense_21_acc: 0.8357 - dense_24_acc: 1.0000 - dense_27_acc: 0.7812 - dense_30_acc: 0.344 - ETA: 5s - loss: 5.8707 - dense_3_loss: 0.0525 - dense_6_loss: 0.0495 - dense_9_loss: 0.8754 - dense_12_loss: 1.8260 - dense_15_loss: 3.1492e-04 - dense_18_loss: 0.0574 - dense_21_loss: 0.5611 - dense_24_loss: 3.3251e-04 - dense_27_loss: 0.6608 - dense_30_loss: 1.7875 - dense_3_acc: 0.9819 - dense_6_acc: 0.9800 - dense_9_acc: 0.5590 - dense_12_acc: 0.2989 - dense_15_acc: 1.0000 - dense_18_acc: 0.9757 - dense_21_acc: 0.8363 - dense_24_acc: 1.0000 - dense_

10000/10000 [==============================] - ETA: 2s - loss: 5.8026 - dense_3_loss: 0.0523 - dense_6_loss: 0.0490 - dense_9_loss: 0.8743 - dense_12_loss: 1.8116 - dense_15_loss: 3.1144e-04 - dense_18_loss: 0.0570 - dense_21_loss: 0.5510 - dense_24_loss: 3.2850e-04 - dense_27_loss: 0.6512 - dense_30_loss: 1.7556 - dense_3_acc: 0.9812 - dense_6_acc: 0.9797 - dense_9_acc: 0.5600 - dense_12_acc: 0.3051 - dense_15_acc: 1.0000 - dense_18_acc: 0.9759 - dense_21_acc: 0.8408 - dense_24_acc: 1.0000 - dense_27_acc: 0.7880 - dense_30_acc: 0.359 - ETA: 2s - loss: 5.7943 - dense_3_loss: 0.0519 - dense_6_loss: 0.0486 - dense_9_loss: 0.8741 - dense_12_loss: 1.8099 - dense_15_loss: 3.1051e-04 - dense_18_loss: 0.0567 - dense_21_loss: 0.5496 - dense_24_loss: 3.2795e-04 - dense_27_loss: 0.6508 - dense_30_loss: 1.7521 - dense_3_acc: 0.9814 - dense_6_acc: 0.9799 - dense_9_acc: 0.5601 - dense_12_acc: 0.3057 - dense_15_acc: 1.0000 - dense_18_acc: 0.9761 - dense_21_acc: 0.8416 - dense_24_acc: 1.0000 - dense_

In [13]:
#model.load_weights('models/model.h5')

In [14]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, human_vocab)
    source1 = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source2 = np.reshape(source1, (1, 30, 37))
    prediction = model.predict([source2, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output))

source: 3 May 1979
output: 2017-03-28
source: 5 April 09
output: 2016-06-08
source: 21th of August 2016
output: 1911-10-10
source: Tue 10 Jul 2007
output: 1970-09-20
source: Saturday May 9 2018
output: 1911-10-10
source: March 3 2001
output: 2017-12-29
source: March 3rd 2001
output: 1970-07-28
source: 1 March 2001
output: 2018-06-08
